In [16]:
import pandas as pd
from datetime import datetime

In [17]:
data =pd.read_csv("../data/processed/openweather_hourly/Chicago_hourly_data.csv")

In [18]:
data[data['Snow (1h)'] != 0]

,Year,Month,Day,Time,Temperature (F),Feels Like (F),Temp Min (F),Temp Max (F),Pressure,Humidity,Wind Speed,Wind Deg,Wind Gust,Clouds All,Rain (1h),Snow (1h),Weather Description
240,2024,11,20,21:00:00,33.37,28.67,31.08,34.25,1009,77,5.01,314,8.01,100,0.0,0.39,light snow
243,2024,11,21,00:00:00,32.88,21.43,30.72,34.03,1007,74,18.99,302,27.00,100,0.0,0.15,light snow
244,2024,11,21,01:00:00,31.78,20.97,29.98,33.13,1007,82,15.99,298,23.00,100,0.0,0.47,light snow
252,2024,11,21,09:00:00,30.40,17.80,28.56,31.62,1006,84,23.00,299,32.99,99,0.0,1.09,snow
253,2024,11,21,10:00:00,29.86,18.45,28.72,31.62,1006,91,16.11,260,26.46,100,0.0,1.89,snow
254,2024,11,21,11:00:00,30.88,20.62,30.02,32.76,1005,94,13.80,260,24.16,100,0.0,2.99,snow
255,2024,11,21,12:00:00,32.38,21.31,30.70,34.09,1003,93,17.27,270,23.02,100,0.0,1.54,heavy snow
256,2024,11,21,13:00:00,33.39,20.79,31.55,34.99,1002,93,28.99,279,38.99,100,0.0,0.92,snow
257,2024,11,21,14:00:00,34.36,22.06,32.52,36.12,1002,94,24.00,266,31.00,100,0.0,0.39,light snow
259,2024,11,21,16:00:00,37.09,27.41,35.29,38.62,1002,94,17.27,310,27.63,100,0.0,0.27,mist


In [19]:
restructured_simple = pd.read_csv("../data/restructured_simple/KORD.csv")
restructured_simple.columns

Index(['YEAR', 'MONTH', 'DAY', 'TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'TAVG'], dtype='object')

In [35]:
def process_weather_data(data_file, hourly_data_file):
    data = pd.read_csv(data_file)
    hourly_data = pd.read_csv(hourly_data_file)

    # Convert the 'Datetime' column in 'hourly_data' to a datetime object and extract Year, Month, Day
    hourly_data['Datetime'] = pd.to_datetime(hourly_data[['Year', 'Month', 'Day']])
    hourly_data['Year'] = hourly_data['Datetime'].dt.year
    hourly_data['Month'] = hourly_data['Datetime'].dt.month
    hourly_data['Day'] = hourly_data['Datetime'].dt.day

    start_date = pd.to_datetime('2023-11-30')
    end_date = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))

    data.loc[
        (data['YEAR'] == 2024) &
        ((data['MONTH'] == 11) & (data['DAY'] >= 15)),
        ['TAVG', 'TMIN', 'TMAX', 'PRCP']
    ] = None  # Replace values of these columns to NA for this date range

    hourly_aggregated = hourly_data.groupby(['Year', 'Month', 'Day']).agg(
        TAVG=('Temperature (F)', 'mean'),  # Average Temperature to TAVG
        TMIN=('Temp Min (F)', 'min'),      # Min Temp to TMIN
        TMAX=('Temp Max (F)', 'max'),      # Max Temp to TMAX
        PRCP=('Rain (1h)', 'sum'),        # Pressure to PRCP
        SNOW=('Snow (1h)', 'sum')
    ).reset_index()

    # Rename the columns in hourly_aggregated to avoid conflict during merge
    hourly_aggregated = hourly_aggregated.rename(columns={
        'TAVG': 'TAVG_new',
        'TMIN': 'TMIN_new',
        'TMAX': 'TMAX_new',
        'PRCP': 'PRCP_new',
        'SNOW': 'SNOW_new'
    })

    # Merge the aggregated hourly data with 'data', replacing NA values with hourly aggregated values
    data = pd.merge(data, hourly_aggregated, how='outer', left_on=['YEAR', 'MONTH', 'DAY'], right_on=['Year', 'Month', 'Day'])

    # Replace the NA values in the columns with the values from the aggregated hourly 
    data['YEAR'] = data['YEAR'].combine_first(data['Year']).astype(int)
    data['MONTH'] = data['MONTH'].combine_first(data['Month']).astype(int)
    data['DAY'] = data['DAY'].combine_first(data['Day']).astype(int)

    data['TAVG'] = data['TAVG_new'].combine_first(data['TAVG']) 
    data['TAVG'] = data['TAVG_new'].combine_first(data['TAVG'])  # Replace NA in 'TAVG' with hourly aggregated value
    data['TMIN'] = data['TMIN_new'].combine_first(data['TMIN'])  # Replace NA in 'TMIN' with hourly aggregated value
    data['TMAX'] = data['TMAX_new'].combine_first(data['TMAX'])  # Replace NA in 'TMAX' with hourly aggregated value
    data['PRCP'] = data['PRCP_new'].combine_first(data['PRCP'])  # Replace NA in 'PRCP' with hourly aggregated value
    data['SNOW'] = data['SNOW_new'].combine_first(data['SNOW'])
    # Drop the extra columns created during the merge (e.g., columns with '_new' suffix)
    
    data = data.drop(columns=[col for col in data.columns if col.endswith('_new') or col in ['Year', 'Month', 'Day']])

    return data

In [36]:
combined = process_weather_data("../data/restructured_simple/KORD.csv","../data/processed/openweather_hourly/Chicago_hourly_data.csv")

In [37]:
combined.tail(20)

,YEAR,MONTH,DAY,TMAX,TMIN,PRCP,SNOW,SNWD,TAVG
24111,2024,11,5,66.92,60.08,325.00,0.00,0.0,64.760000
24112,2024,11,6,60.08,48.92,0.00,0.00,0.0,55.220000
24113,2024,11,7,59.00,44.06,0.00,0.00,0.0,51.080000
24114,2024,11,8,62.06,44.06,0.00,0.00,0.0,51.260000
24115,2024,11,9,55.94,44.96,51.00,0.00,0.0,50.900000
24116,2024,11,10,54.12,50.09,0.00,0.00,0.0,52.453333
24117,2024,11,11,59.04,39.61,0.00,0.00,0.0,50.990000
24118,2024,11,12,51.49,39.29,0.00,0.00,NaN,45.706667
24119,2024,11,13,53.49,36.68,6.75,0.00,NaN,46.242083
24120,2024,11,14,54.54,46.67,5.70,0.00,NaN,50.855417
